In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline

In [14]:
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4209, 378)
Shape test: (4209, 377)


In [15]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y'], axis=1).values
X_test= test.values

In [16]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(X)

5

In [22]:
param_grid={'estimator__gamma': [0.1,0.5,1.0],
            'estimator__learning_rate':[0.01,0.05,0.1],
            'estimator__max_depth':[3,5,7,9],
            'estimator__subsample':[0.6,0.8,1.0],
            'estimator__reg_lambda':[0.1,1.0],
            'estimator__reg_alpha':[0,0.5,1.0],
           }

In [23]:
est= XGBRegressor(n_estimators=500, base_score=y_mean, silent=False, n_jobs=1)
selector= RFECV(est, step=5, cv=5, n_jobs=1)
clf=GridSearchCV(selector, param_grid = param_grid ,scoring='r2', cv=kf, n_jobs =1)

In [24]:
clf.fit(X,y)

KeyboardInterrupt: 

In [5]:
for train_index, test_index in kf.split(X):
    X_train, y_train=X[train_index], y[train_index]
    X_valid, y_valid=X[test_index], y[test_index]
    
    estimator = XGBRegressor(base_score=100.66931812782134, booster='gbtree',
       colsample_bylevel=1, colsample_bytree=1, gamma=1.0,
       learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=2,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1.0,
       reg_lambda=1.0, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.8)
    
    selector = RFECV(estimator, step=5, cv=5, n_jobs=2)
    selector.fit(X_train, y_train)
    print("R^2 valid: {: }".format(r2_score(y_valid, selector.predict(X_valid))))
    print("R^2 train: {: }".format(r2_score(y_train, selector.predict(X_train))))
   

In [6]:
#estimator = XGBRegressor(base_score=100.66931812782134, booster='gbtree',
#       colsample_bylevel=1, colsample_bytree=1, gamma=1.0,
#       learning_rate=0.01, max_delta_step=0, max_depth=3,
#       min_child_weight=1, missing=None, n_estimators=500, n_jobs=2,
#       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1.0,
#       reg_lambda=1.0, scale_pos_weight=1, seed=None, silent=False,
#       subsample=0.8)

In [7]:
#selector= RFE(estimator, n_features_to_select=300, step=5)
#selector = RFECV(estimator, step=5, cv=5, n_jobs=2)

In [8]:
#selector.fit(X_train, y_train)

RFECV(cv=5,
   estimator=XGBRegressor(base_score=100.66931812782134, booster='gbtree',
       colsample_bylevel=1, colsample_bytree=1, gamma=1.0,
       learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, n_jobs=2,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=1.0,
       reg_lambda=1.0, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.8),
   n_jobs=2, scoring=None, step=5, verbose=0)

In [22]:
#selector.n_features_

97

In [9]:
print("R^2 valid: {: }".format(r2_score(y_valid, selector.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, selector.predict(X_train))))

R^2 valid:  0.5932869296551689
R^2 train:  0.6038073891312494


In [10]:
y_pred=selector.predict(X_test)

In [11]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred
sub.to_csv('xgb_RFE-best.csv', index=False)